# Music Genre Classification- Spotify Factors for Popularity

We plan on using our data set to determine what traits make a song popular in regards to loudness(dB), energy levels, and beats per minute. We will also use this to determine if there is a common relationship between genres. We will do that by making several different plots and types of plots to show the correlations over the time frame of 2010-2019.


Which factor affects popularity the most?

    -Song popularity vs. dB
    -Song popularity vs. energy levels
    -Song popularity vs. danceability 
    -Song popularity vs. likelihood the song is live or not
    -Song popularity vs. BPM


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.datasets import fetch_lfw_people
import time
from sklearn.datasets import make_circles
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split

In [27]:
data = pd.read_csv("top10s.csv")

data

,Unnamed: 0,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,599,Find U Again (feat. Camila Cabello),Mark Ronson,dance pop,2019,104,66,61,-7,20,16,176,1,3,75
599,600,Cross Me (feat. Chance the Rapper & PnB Rock),Ed Sheeran,pop,2019,95,79,75,-6,7,61,206,21,12,75
600,601,"No Brainer (feat. Justin Bieber, Chance the Ra...",DJ Khaled,dance pop,2019,136,76,53,-5,9,65,260,7,34,70
601,602,Nothing Breaks Like a Heart (feat. Miley Cyrus),Mark Ronson,dance pop,2019,114,79,60,-6,42,24,217,1,7,69


In [29]:
#drop unnamed column

data.drop(["Unnamed: 0"], axis = 1)

,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,Find U Again (feat. Camila Cabello),Mark Ronson,dance pop,2019,104,66,61,-7,20,16,176,1,3,75
599,Cross Me (feat. Chance the Rapper & PnB Rock),Ed Sheeran,pop,2019,95,79,75,-6,7,61,206,21,12,75
600,"No Brainer (feat. Justin Bieber, Chance the Ra...",DJ Khaled,dance pop,2019,136,76,53,-5,9,65,260,7,34,70
601,Nothing Breaks Like a Heart (feat. Miley Cyrus),Mark Ronson,dance pop,2019,114,79,60,-6,42,24,217,1,7,69


 All of the variables and what they mean: 
 
    - title = title of the song
    - artist = song's artist
    - top genre = genre of the track
    - year = song's year in the Billboard
    - bpm = beats per minute or the tempo of the song
    - nrgy = energy or the energy of a song, calculated through a higher the value, the more energetic the song is
    - dnce = dancability meaning the higher the value the easier it is to dance to 
    - dB = loudness, the higher the value the louder it is 
    - live = liveliness, the higher the value the more likely the song is a live recording
    - val = calence, the higher it is, the more positive mood for the song
    - dur = the length of the song
    - acous = the more accoustic the song is
    - spch = the more spoken word the song contains
    - pop = the higher, the more popular the song is

# Song Popularity vs. dB

In [39]:
#data2 = np.arange(603).reshape((-1,1))


In [44]:
train_vectors, test_vectors,train_labels, test_labels = train_test_split(data["pop"], data["bpm"], test_size=0.25, random_state=420)


In [45]:
start = time.time()

tmp_vectors = train_vectors
tmp_labels = train_labels

print("Fitting the classifier to the training set")

param_grid = {'C': [1e3, 5e3, 1e4, 5e4],
               'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
               'kernel': ['linear','rbf']}
clf = GridSearchCV(SVC(class_weight='balanced'), param_grid)


clf = clf.fit(tmp_vectors, tmp_labels)
print("Best estimator found by grid search:")
print(clf.best_estimator_)
print("Best parameters found by grid search:")
print(clf.best_params_)

end = time.time()
print("Runtime",end - start)

Fitting the classifier to the training set


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 169, in fit
    X, y = self._validate_data(X, y, dtype=np.float64,
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sk

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 169, in fit
    X, y = self._validate_data(X, y, dtype=np.float64,
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 814, in check_X_y
    X = check_array(X, accept_sparse=accept_sparse,
  Fil

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 169, in fit
    X, y = self._validate_data(X, y, dtype=np.float64,
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 814, in check_X_y
    X = check_array(X, accept_sparse=accept_sparse,
  Fil

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 169, in fit
    X, y = self._validate_data(X, y, dtype=np.float64,
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 814, in check_X_y
    X = check_array(X, accept_sparse=accept_sparse,
  Fil

ValueError: Expected 2D array, got 1D array instead:
array=[83. 72. 72. 70. 41. 53. 71. 70. 77. 59. 77. 84. 59. 73. 61. 59. 60. 45.
 35. 30. 81. 60. 70. 65. 86. 49. 28. 62. 68. 52. 74. 56. 68. 58. 72. 59.
 72. 63. 72. 69. 86. 56. 72. 53. 46. 67. 84. 80. 73. 60. 45. 57. 41. 69.
 65. 47. 85. 79. 66. 62. 85. 83. 58. 57. 82. 38. 90. 73. 76. 66. 86. 65.
 74. 78. 73. 46. 44. 78. 84. 64. 76. 69. 57. 68. 75. 36. 56. 65. 67. 76.
 66. 63. 76. 70. 54. 52. 61. 75. 64. 79. 76. 61. 71. 78. 76. 67. 72. 79.
 61. 54. 72.  0. 64. 71. 60. 74. 70. 57. 75. 78. 57. 65. 71. 67. 72. 80.
 69. 76. 81. 97. 70. 92. 76. 78. 68. 83. 52. 80. 76. 99. 84. 52. 78. 57.
 76. 38. 70. 69. 65. 43. 81. 76. 46. 81. 62. 66. 85. 74. 73. 69. 70. 42.
 69. 54. 77. 68. 84. 77. 58. 82. 70. 65. 71. 50. 72. 77. 83. 65. 72. 66.
 60. 79. 18. 65. 52. 80. 68. 72. 52. 76. 38. 78. 86. 64. 79. 88. 62. 41.
 67. 78. 79. 74. 68. 64. 71. 62. 67. 57. 61. 61. 58. 69. 83. 80. 64. 72.
 78. 51. 80. 73. 82. 76. 79. 69. 77. 56. 59. 44. 80. 92. 59. 73. 

error states "Expected 2D array, got 1D array"

In [43]:
predict_vectors = test_vectors
true_labels = test_labels

print("Predicting names on the test set")
pred_labels = clf.predict(predict_vectors)

print(classification_report(true_labels, pred_labels))
print(confusion_matrix(true_labels, pred_labels, labels=range(n_classes)))


Predicting names on the test set


NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.